# Import das Bibliotecas

In [1]:
import pandas as pd
import numpy as np

#utils
import emoji, re, string, time, os

#nlp
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

# Carregamento do dataset

In [2]:
original_df = pd.read_csv("C:/Users/lmb3/source/repos/FakeWhatsApp.Br-master/data/2018/fakeWhatsApp.BR_2018.csv")

C:\Users\lmb3\anaconda3\envs\pycaret\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
original_df = original_df[original_df['midia']==0]
original_df = original_df[original_df['viral']==1]
original_df.shape

(20872, 19)

In [4]:
original_df = original_df.drop_duplicates(subset=['text'])
original_df.shape

(5284, 19)

In [5]:
text_df = original_df[["text","misinformation"]]

In [6]:
text_df["misinformation"].value_counts()

 0    2547
 1    2041
-1     696
Name: misinformation, dtype: int64

In [7]:
text_df = text_df[text_df["misinformation"]>-1]
text_df.reset_index(inplace=True)
text_df = text_df.drop('index',axis=1)
text_df

,text,misinformation
0,"No dia 07 de Outubro, quando for votar, não es...",1
1,Devido a aborrecimentos nas varias redes socia...,0
2,EU GOSTARIA DE PEDIR A TODOS DO GRUPOS ENTREM ...,0
3,Como são as coisas.\nChefe do jacaré aparece n...,1
4,A ratoeira funcionou mais a ratazana aguentou ...,0
...,...,...
4583,Amoedo - Itaú - Metacapitalistas (George Soros...,1
4584,*BEM GALERA... O VIDEO DA PSICOLOGIA REVERSA P...,0
4585,DIVULGUEM ESSE VIDEO PARA QUE TODOS SAIBAM QUE...,1
4586,https://www.oantagonista.com/brasil/exclusivo-...,0


In [8]:
text_df.to_csv('data/original_dataset.csv')

# Nesta sequencia serão preparados diferentes tipos de pré-processamento

**Do que foi feito pelo autor, sera aplicado o filtro para extrair o dominio da url dos links e reduzir todas as risadas com 'k' para 'kkkk'. Alem disso, todas as letras serao tratadas como minusculas**

## Funções

**Aqui serão inicializadas as funções que serão usadas em cada pré processamento**

In [9]:
def domainUrl(text):
    '''
    ORIGINAL DO ARTIGO DE Cabral et al. 2021
    Substitutes an URL in a text for the domain of this URL
    Input: an string
    Output: the string with the modified URL
    '''    
    if 'http' in text:
        re_url = '[^\s]*https*://[^\s]*'
        matches = re.findall(re_url, text, flags=re.IGNORECASE)
        for m in matches:
            domain = m.split('//')
            domain = domain[1].split('/')[0]
            text = re.sub(re_url, domain, text, 1)
        return text
    else:
        return text

def processLoL(text):
    '''
    ORIGINAL DO AUTOR DE Cabral et al. 2021
    substitutes any laughthing with k for kkkk
    Input: an string
    Output: the string with the modified laghter
    '''
    re_kkk = 'kkk*'
    t = re.sub(re_kkk, "kkkk", text, flags=re.IGNORECASE)
    return t

def KeepEmojisPunctuation(text):
    '''
    ADAPTADO DO AUTOR DE Cabral et al. 2021
    Put spaces between emojis and punctuation.
    '''
    
    emojis_list = list(emoji.UNICODE_EMOJI['pt'].keys()) + ['🛇'] + ['☆']
    punct = list(string.punctuation) + ['\n']
    
    #get all unique chars
    chars = set(text)
    #for each unique char in text, do:
    for c in chars:
            
        if c in punct: 
            text = text.replace(c, ' ' + c + ' ')    
        
        if c in emojis_list:
            text = text.replace(c, ' ' + c + ' ') 
            
    text = re.sub(' +', ' ', text)
    return text
    
def RemovePunctuation_KeepEmojis(text):
    '''
    ADAPTADO DO AUTOR DE Cabral et al. 2021
    Put spaces between emojis. Removes punctuation.
    '''
    
    emojis_list = list(emoji.UNICODE_EMOJI['pt'].keys()) + ['🛇'] + ['☆']
    punct = list(string.punctuation) + ['\n']
    
    #get all unique chars
    chars = set(text)
    #for each unique char in text, do:
    for c in chars:
        if c in punct: 
            text = text.replace(c, ' ') #Changes ponctuation for space
        
        if c in emojis_list:
            text = text.replace(c, ' ' + c + ' ') # puts emoji between spaces                 
            
    text = re.sub(' +', ' ', text)
    return text

def RemoveEmojis_KeepPunctuation(text):
    '''
    ADAPTADO DO AUTOR DE Cabral et al. 2021
    Put spaces between punctuation. Removes emojis.
    '''
    
    emojis_list = list(emoji.UNICODE_EMOJI['pt'].keys()) + ['🛇'] + ['☆']
    punct = list(string.punctuation) + ['\n']
    
    #get all unique chars
    chars = set(text)
    #for each unique char in text, do:
    for c in chars:
        if c in punct: 
            text = text.replace(c, ' ' + c + ' ')  # puts ponctuation between spaces   
        
        if c in emojis_list:
            text = text.replace(c, ' ') #Changes emoji for space
            
    text = re.sub(' +', ' ', text)
    return text

def RemovePunctuationEmojis(text):
    '''
    Put spaces between emojis. Removes punctuation.
    Removes emojis and punctuation.
    '''
    
    emojis_list = list(emoji.UNICODE_EMOJI['pt'].keys()) + ['🛇'] + ['☆']
    punct = list(string.punctuation) + ['\n']
    
    #get all unique chars
    chars = set(text)
    #for each unique char in text, do:
    for c in chars:
        if c in punct: 
            text = text.replace(c, ' ') #Changes punctuation for space
            
        if c in emojis_list:
            text = text.replace(c, ' ') #Changes emoji for space
            
    text = re.sub(' +', ' ', text)
    return text




#List of stopwords in portuguese form NLTK
stop_words = list(stopwords.words('portuguese'))
#New list of words added by Cabral et al. 2021
new_stopwords = ['aí','pra','vão','vou','onde','lá','aqui',
                 'tá','pode','pois','so','deu','agora','todo',
                 'nao','ja','vc', 'bom', 'ai','ta', 'voce', 'alguem', 'ne', 'pq',
                 'cara','to','mim','la','vcs','tbm', 'tudo']
#Uniting the lists
stop_words = stop_words + new_stopwords
#Transforming the list in a srting with words separated by whitespaces
final_stop_words = []
for sw in stop_words:
    sw = ' '+ sw + ' '
    final_stop_words.append(sw)
    
def removeStopwords(text):
    '''
    ORIGINAL DO ARTIGO DE Cabral et al. 2021
    Substitutes an stopword with a whitespace
    Input: an string
    Output: the string without stopwords
    '''  
    for sw in final_stop_words:
        text = text.replace(sw,' ')
    text = re.sub(' +',' ',text)
    return text



def preprocess1(text):
    '''
    Main function for pre-processing 1
    '''
    text = text.lower().strip()
    text = domainUrl(text)
    text = processLoL(text)
    text = KeepEmojisPunctuation(text)

    return text

def preprocess2(text):
    '''
    Main function for pre-processing 2
    '''
    text = text.lower().strip()
    text = domainUrl(text)
    text = processLoL(text)
    text = KeepEmojisPunctuation(text)
    text = removeStopwords(text)

    return text

def preprocess3(text):
    '''
    Main function for pre-processing 3
    '''
    text = text.lower().strip()
    text = domainUrl(text)
    text = processLoL(text)
    text = RemovePunctuation_KeepEmojis(text)
    text = removeStopwords(text)

    return text

def preprocess4(text):
    '''
    Main function for pre-processing 4
    '''
    text = text.lower().strip()
    text = domainUrl(text)
    text = processLoL(text)
    text = RemoveEmojis_KeepPunctuation(text)
    text = removeStopwords(text)

    return text

def preprocess5(text):
    '''
    Main function for pre-processing 5
    '''
    text = text.lower().strip()
    text = domainUrl(text)
    text = processLoL(text)
    text = RemovePunctuationEmojis(text)

    return text

def preprocess6(text):
    '''
    Main function for pre-processing 6
    '''
    text = text.lower().strip()
    text = domainUrl(text)
    text = processLoL(text)
    text = RemovePunctuationEmojis(text)
    text = removeStopwords(text)

    return text

# Pre-Procesamento 1: Mantidos stopwords, pontuações e emojis

In [10]:
text_df['pre_processed_text'] = [preprocess1(t) for t in text_df['text']]

In [11]:
text_df['pre_processed_text']

0       no dia 07 de outubro , quando for votar , não ...
1       devido a aborrecimentos nas varias redes socia...
2       eu gostaria de pedir a todos do grupos entrem ...
3       como são as coisas . \n chefe do jacaré aparec...
4       a ratoeira funcionou mais a ratazana aguentou ...
                              ...                        
4583    amoedo - itaú - metacapitalistas ( george soro...
4584     * bem galera . . . o video da psicologia reve...
4585    divulguem esse video para que todos saibam que...
4586    www . oantagonista . com * exclusivo : bolsona...
4587     * # bolsonaro2018 * 🇧🇷 \n * 1 * - o foco é tr...
Name: pre_processed_text, Length: 4588, dtype: object

In [12]:
text_df.to_csv('data_processed/stopwordPunctEmojiKeep_dataset.csv')

# Pre-Processamento 2: Remoção de stopwords, são mantidas pontuações e emojis

In [13]:
text_df['pre_processed_text'] = [preprocess2(t) for t in text_df['text']]

In [14]:
text_df['pre_processed_text']

0       no dia 07 outubro , votar , esqueça esquerdopa...
1       devido aborrecimentos varias redes sociais pos...
2       eu gostaria pedir todos grupos entrem neste li...
3       como coisas . \n chefe jacaré aparece video ba...
4       a ratoeira funcionou ratazana aguentou dor cal...
                              ...                        
4583    amoedo - itaú - metacapitalistas ( george soro...
4584     * bem galera . . . video psicologia reversa a...
4585    divulguem video todos saibam bolsonaro . . . 😑...
4586    www . oantagonista . * exclusivo : bolsonaro f...
4587     * # bolsonaro2018 * 🇧🇷 \n * 1 * - foco trocar...
Name: pre_processed_text, Length: 4588, dtype: object

In [15]:
text_df.to_csv('data_processed/stopwordRemoval_punctEmojiKeep_dataset.csv')

# Pre-Processamento 3: Remoção de stopwords e de pontuação, emojis mantidos

In [16]:
text_df['pre_processed_text'] = [preprocess3(t) for t in text_df['text']]

In [17]:
text_df['pre_processed_text']

0       no dia 07 outubro votar esqueça esquerdopatas ...
1       devido aborrecimentos varias redes sociais pos...
2       eu gostaria pedir todos grupos entrem neste li...
3       como coisas chefe jacaré aparece video baleado...
4       a ratoeira funcionou ratazana aguentou dor cal...
                              ...                        
4583    amoedo itaú metacapitalistas george soros gove...
4584     bem galera video psicologia reversa ajudar pr...
4585    divulguem video todos saibam bolsonaro 😑 🇧🇷 👇 ...
4586    www oantagonista exclusivo bolsonaro faz nova ...
4587     bolsonaro2018 🇧🇷 1 foco trocar informações po...
Name: pre_processed_text, Length: 4588, dtype: object

In [18]:
text_df.to_csv('data_processed/punctStopwordRemoval_emojiKeep_dataset.csv')

# Pre-Processamento 4: Remoção de stopwords e de emojis, pontuação mantida

In [19]:
text_df['pre_processed_text'] = [preprocess4(t) for t in text_df['text']]

In [20]:
text_df['pre_processed_text']

0       no dia 07 outubro , votar , esqueça esquerdopa...
1       devido aborrecimentos varias redes sociais pos...
2       eu gostaria pedir todos grupos entrem neste li...
3       como coisas . \n chefe jacaré aparece video ba...
4       a ratoeira funcionou ratazana aguentou dor cal...
                              ...                        
4583    amoedo - itaú - metacapitalistas ( george soro...
4584     * bem galera . . . video psicologia reversa a...
4585    divulguem video todos saibam bolsonaro . . . 🇧...
4586    www . oantagonista . * exclusivo : bolsonaro f...
4587     * # bolsonaro2018 * 🇧🇷 \n * 1 * - foco trocar...
Name: pre_processed_text, Length: 4588, dtype: object

In [21]:
text_df.to_csv('data_processed/stopwordEmojiRemoval_punctKeep_dataset.csv')

# Pre-Processamento 5: Remoção de emojis e pontuações, stopwords mantidas 

In [22]:
text_df['pre_processed_text'] = [preprocess5(t) for t in text_df['text']]

In [23]:
text_df['pre_processed_text']

0       no dia 07 de outubro quando for votar não esqu...
1       devido a aborrecimentos nas varias redes socia...
2       eu gostaria de pedir a todos do grupos entrem ...
3       como são as coisas chefe do jacaré aparece no ...
4       a ratoeira funcionou mais a ratazana aguentou ...
                              ...                        
4583    amoedo itaú metacapitalistas george soros no g...
4584     bem galera o video da psicologia reversa para...
4585    divulguem esse video para que todos saibam que...
4586    www oantagonista com exclusivo bolsonaro faz n...
4587     bolsonaro2018 🇧🇷 1 o foco é trocar informaçõe...
Name: pre_processed_text, Length: 4588, dtype: object

In [24]:
text_df.to_csv('data_processed/punctEmojiRemoval_stopwordKeep_dataset.csv')

# Pre-Processamento 6: Remoção de stopwords, emojis e pontuções

In [25]:
text_df['pre_processed_text'] = [preprocess6(t) for t in text_df['text']]

In [26]:
text_df['pre_processed_text']

0       no dia 07 outubro votar esqueça esquerdopatas ...
1       devido aborrecimentos varias redes sociais pos...
2       eu gostaria pedir todos grupos entrem neste li...
3       como coisas chefe jacaré aparece video baleado...
4       a ratoeira funcionou ratazana aguentou dor cal...
                              ...                        
4583    amoedo itaú metacapitalistas george soros gove...
4584     bem galera video psicologia reversa ajudar pr...
4585           divulguem video todos saibam bolsonaro 🇧🇷 
4586    www oantagonista exclusivo bolsonaro faz nova ...
4587     bolsonaro2018 🇧🇷 1 foco trocar informações po...
Name: pre_processed_text, Length: 4588, dtype: object

In [27]:
text_df.to_csv('data_processed/stopwordPuncEmojitRemoval_dataset.csv')